In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code ANWXBJQPS to authenticate.
You have logged in. Now let us find all the subscriptions to which you have access...
Interactive authentication successfully completed.
Workspace name: quick-starts-ws-174722
Azure region: southcentralus
Subscription id: 48a74bb7-9950-4cc1-9caa-5d50f995cc55
Resource group: aml-quickstarts-174722


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
cpu_cluster_name = 'EduardoCluster'
compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',max_nodes=4)
cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

In [3]:
from azureml.core import Environment

#sklearn_env = Environment.from_conda_specification(name = 'sklearn-env', file_path= './conda_dependencies.yml')
sklearn_env = Environment.get(workspace=ws, name="AzureML-sklearn-0.24-ubuntu18.04-py37-cpu")
from azureml.core import ScriptRunConfig
src = ScriptRunConfig(source_directory='./',compute_target=cpu_cluster_name,script='train.py',environment=sklearn_env)

In [7]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform,choice
import os

# Specify parameter sampler
ps = RandomParameterSampling({'--C':choice(0.001,0.1,1,10,20,30,40),'--max_iter':choice(10,20,50,100,150,200)})

# Specify a Policy
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
est = src

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(run_config=est,policy=policy,hyperparameter_sampling=ps,primary_metric_name='Accuracy',primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,max_total_runs=12)


In [8]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###
hyperdrive_run = exp.submit(hyperdrive_config)

In [18]:
# Show the best run
best_hyperdriverun = hyperdrive_run.get_best_run_by_primary_metric()
best_hyperdriverun.get_details()['runDefinition']['arguments']

['--C', '0.1', '--max_iter', '100']

In [19]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###
model = best_hyperdriverun.register_model(model_name = 'best_hyperdrive_model', model_path = 'outputs/model.joblib')

In [9]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
path ="https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

ds = TabularDatasetFactory.from_delimited_files(path)

In [10]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

In [11]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=ds,
    label_column_name='y',
    n_cross_validations=2)

In [12]:
# Submit your automl run

### YOUR CODE HERE ###
autoML_run = exp.submit(automl_config, show_output = False)

2022-01-12:08:36:55,172 INFO     [modeling_bert.py:226] Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .
2022-01-12:08:36:55,202 INFO     [modeling_xlnet.py:339] Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .
2022-01-12:08:37:28,839 INFO     [utils.py:157] NumExpr defaulting to 4 threads.
2022-01-12:08:37:29,613 WARNING  [local_experiment_driver.py:148] Running on local machine. Note that local runs always run synchronously even if you use the parameter 'show_output=False'


Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,AutoML_35ee3919-9a19-4c8d-bb0b-51e82b7227fd,automl,Preparing,Link to Azure Machine Learning studio,Link to Documentation


2022-01-12:08:58:56,855 INFO     [explanation_client.py:332] Using default datastore for uploads
2022-01-12:08:59:03,22 WARNING  [explanation_utils.py:92] Cannot serialize JSON possibly due to timestamp, retrying


In [24]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###
best_autoML_run, best_autmoML_model = autoML_run.get_output()
autoML_model = best_autoML_run.register_model(model_name = 'best_autoML_model.pkl', model_path = './outputs/')

In [25]:
autoML_run.get_output()

(Run(Experiment: udacity-project,
 Id: AutoML_35ee3919-9a19-4c8d-bb0b-51e82b7227fd_30,
 Type: None,
 Status: Completed),
 PipelineWithYTransformations(Pipeline={'memory': None,
                                        'steps': [('datatransformer',
                                                   DataTransformer(enable_dnn=False, enable_feature_sweeping=True, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=True, is_onnx_compatible=False, observer=None, task='classification', working_dir='/mn...
 )), ('logisticregression', LogisticRegression(C=51.79474679231202, class_weight=None, dual=False, fit_intercept=True, intercept_scaling=1, l1_ratio=None, max_iter=100, multi_class='ovr', n_jobs=1, penalty='l2', random_state=None, solver='lbfgs', tol=0.0001, verbose=0, warm_start=False))], verbose=False)), ('13', Pipeline(memory=None, steps=[('maxabsscaler', MaxAbsScaler(copy=True)), ('sgdclassifierwrapper', SGDClas

In [26]:
x

,age,marital,default,housing,loan,month,day_of_week,duration,campaign,pdays,...,contact_cellular,contact_telephone,education_basic.4y,education_basic.6y,education_basic.9y,education_high.school,education_illiterate,education_professional.course,education_university.degree,education_unknown
0,57,1,0,0,1,5,1,371,1,999,...,1,0,0,0,0,1,0,0,0,0
1,55,1,0,1,0,5,4,285,2,999,...,0,1,0,0,0,0,0,0,0,1
2,33,1,0,0,0,5,5,52,1,999,...,1,0,0,0,1,0,0,0,0,0
3,36,1,0,0,0,6,5,355,4,999,...,0,1,0,0,0,1,0,0,0,0
4,27,1,0,1,0,7,5,189,2,999,...,1,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32945,56,1,0,0,1,7,1,116,1,999,...,1,0,1,0,0,0,0,0,0,0
32946,37,1,0,0,1,7,5,69,7,999,...,1,0,0,0,0,0,0,0,1,0
32947,26,0,0,0,0,5,2,135,4,999,...,1,0,0,0,0,0,0,0,1,0
32948,31,0,0,0,0,4,1,386,1,999,...,1,0,0,0,1,0,0,0,0,0


In [27]:
y

0        0
1        0
2        0
3        0
4        0
        ..
32945    0
32946    0
32947    0
32948    0
32949    0
Name: y, Length: 32950, dtype: int64